In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
def sentiment_to_numeric(sentiment):
    if sentiment.lower() == 'positive':
        return 1
    elif sentiment.lower() == 'negative':
        return 0
    else:
        raise ValueError("Invalid sentiment. Only 'positive' and 'negative' are allowed.")

In [ ]:
def convert_sentiment_column_to_numeric(df, column_name):
    df[column_name] = df[column_name].apply(sentiment_to_numeric)

In [ ]:
convert_sentiment_column_to_numeric(df, 'sentiment')

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
Vect=CountVectorizer()
docs = np.array(['This is the first document.',
       'This document is the second document.',
       'And this is the third one.',
       'Is this the first document?'])
bag=Vect.fit_transform(docs)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)
tfidf=TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
print(tfidf.fit_transform(bag).toarray())

[[0.   0.47 0.58 0.38 0.   0.   0.38 0.   0.38]
 [0.   0.69 0.   0.28 0.   0.54 0.28 0.   0.28]
 [0.51 0.   0.   0.27 0.51 0.   0.27 0.51 0.27]
 [0.   0.47 0.58 0.38 0.   0.   0.38 0.   0.38]]


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(
                      use_idf=True,
                      norm='l2',
                      smooth_idf=True)

In [ ]:
y=df.sentiment.values
X=tfidf.fit_transform(df['review'].values.astype('U'))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1,test_size=0.5,shuffle=False)

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=5,scoring='accuracy',
                        random_state=0,
                        n_jobs=-1,
                        verbose=3,
                        max_iter=300).fit(X_train,y_train)

saved_model=open('saved_model.sav', 'wb')
pickle.dump(clf,saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.4min finished


In [ ]:
filename='saved_model.sav'
saved_clf=pickle.load(open(filename,'rb'))

saved_clf.score(X_test,y_test)

0.89712

In [ ]:
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

In [ ]:
    def analyze_sentiment(review):
        review_vectorized = tfidf.transform([review])
        prediction = saved_clf.predict(review_vectorized)
        if prediction[0]==1:
            return 'Positive'
        else:
            return 'Negative'

In [ ]:
tfidf = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True)

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")
X_train = tfidf.fit_transform(df['review'].values.astype('U'))

In [ ]:
user_review = input("Enter your movie review: ")

Enter your movie review: not worth watching


In [ ]:
result = analyze_sentiment(user_review)
print("Sentiment Analysis Result: ", result)

Sentiment Analysis Result:  Negative


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
import pickle

drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv")

def convert_sentiment_to_numeric(sentiment):
    if sentiment.lower() == 'positive':
        return 1
    elif sentiment.lower() == 'negative':
        return 0
    else:
        raise ValueError("Invalid sentiment. Only 'positive' and 'negative' are allowed.")

def convert_sentiment_column_to_numeric(df, column_name):
    df[column_name] = df[column_name].apply(convert_sentiment_to_numeric)

convert_sentiment_column_to_numeric(df, 'sentiment')

tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True)
X = tfidf_vectorizer.fit_transform(df['review'].values.astype('U'))
y = df.sentiment.values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5, shuffle=False)

clf = LogisticRegressionCV(cv=5, scoring='accuracy', random_state=0, n_jobs=-1, verbose=3, max_iter=300)
clf.fit(X_train, y_train)

saved_model_file = open('saved_model.pkl', 'wb')
pickle.dump(clf, saved_model_file)
saved_model_file.close()

filename = 'saved_model.pkl'
saved_clf = pickle.load(open(filename, 'rb'))

def analyze_sentiment(review):
    review_vectorized = tfidf_vectorizer.transform([review])
    prediction = saved_clf.predict(review_vectorized)
    if prediction[0] == 1:
        return 'Positive'
    else:
        return 'Negative'

user_review = input("Enter your movie review: ")
result = analyze_sentiment(user_review)
print("Sentiment Analysis Result: ", result)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.9min finished


Enter your movie review: good movie
Sentiment Analysis Result:  Positive
